In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_2.csv")

In [3]:
df.head()

,timestamp_local,temp,city_name,country_code,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,Gujrāt,PK,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,Gujrāt,PK,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,Gujrāt,PK,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,Gujrāt,PK,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,Gujrāt,PK,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
data = df[['o3']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled = scaled.reshape((19706,))

In [8]:
scaled.shape

(19706,)

In [9]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [10]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [11]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [12]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 32)             │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 5312)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         892,584 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 892,712 (3.41 MB)

 Trainable params: 892,712 (3.41 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - loss: 0.0115 - root_mean_squared_error: 0.1040 - val_loss: 0.0064 - val_root_mean_squared_error: 0.0800 - learning_rate: 0.0010
Epoch 2/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - loss: 0.0062 - root_mean_squared_error: 0.0790 - val_loss: 0.0061 - val_root_mean_squared_error: 0.0783 - learning_rate: 0.0010
Epoch 3/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0059 - root_mean_squared_error: 0.0770 - val_loss: 0.0059 - val_root_mean_squared_error: 0.0768 - learning_rate: 0.0010
Epoch 4/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 0.0058 - root_mean_squared_error: 0.0763 - val_loss: 0.0060 - val_root_mean_squared_error: 0.0771 - learning_rate: 0.0010
Epoch 5/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.0058 - root_mean_squared_error: 0.0760 - val_loss: 0.0059 - val_root_mean_squared_error: 0.0770 - learning_rate: 0.0010
Epoch 6/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 0.0056 - root_mean_squa

In [13]:
forecast = model.predict(X_test)

122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [14]:
poll = np.array(df["o3"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 3.9566438


In [17]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 3.2800547778606415


In [18]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 96.71994522213936 %


In [27]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 3.026497


In [19]:
y_test_true.shape

(3874, 168)

In [20]:
testPredict.shape

(3874, 168)

In [21]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [22]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 0.9831
RMSE for segment 2: 1.7514
RMSE for segment 3: 2.3954
RMSE for segment 4: 2.7624
RMSE for segment 5: 3.0222
RMSE for segment 6: 3.1956
RMSE for segment 7: 3.2774
RMSE for segment 8: 3.3093
RMSE for segment 9: 3.3432
RMSE for segment 10: 3.4057
RMSE for segment 11: 3.4194
RMSE for segment 12: 3.4315
RMSE for segment 13: 3.4452
RMSE for segment 14: 3.4351
RMSE for segment 15: 3.4268
RMSE for segment 16: 3.4399
RMSE for segment 17: 3.4607
RMSE for segment 18: 3.5185
RMSE for segment 19: 3.5505
RMSE for segment 20: 3.5434
RMSE for segment 21: 3.5355
RMSE for segment 22: 3.5153
RMSE for segment 23: 3.4875
RMSE for segment 24: 3.4840
RMSE for segment 25: 3.5395
RMSE for segment 26: 3.5812
RMSE for segment 27: 3.6646
RMSE for segment 28: 3.7387
RMSE for segment 29: 3.7920
RMSE for segment 30: 3.8202
RMSE for segment 31: 3.8273
RMSE for segment 32: 3.8086
RMSE for segment 33: 3.7886
RMSE for segment 34: 3.7765
RMSE for segment 35: 3.8099
RMSE for segment 36: 3.8865
R

In [23]:
forecast

array([[0.16116372, 0.16638248, 0.16893211, ..., 0.14472392, 0.15472215,
        0.15930325],
       [0.5528743 , 0.5631961 , 0.5607244 , ..., 0.5380846 , 0.5216266 ,
        0.5132997 ],
       [0.32481074, 0.3883506 , 0.44992256, ..., 0.1900531 , 0.22161554,
        0.25979862],
       ...,
       [0.6678637 , 0.5860373 , 0.5040754 , ..., 0.39490664, 0.40893912,
        0.43255836],
       [0.06558615, 0.09426196, 0.29270643, ..., 0.05516756, 0.0600818 ,
        0.07570019],
       [0.18669437, 0.16360666, 0.14268562, ..., 0.3577841 , 0.29316005,
        0.23115674]], dtype=float32)

In [24]:
testPredict

array([[ 87.04286 ,  87.32649 ,  87.46506 , ...,  86.14938 ,  86.69277 ,
         86.94175 ],
       [108.33164 , 108.89261 , 108.75828 , ..., 107.52785 , 106.63338 ,
        106.18083 ],
       [ 95.93678 ,  99.39007 , 102.73639 , ...,  88.612946,  90.32831 ,
         92.403496],
       ...,
       [114.581116, 110.13399 , 105.679504, ...,  99.746376, 100.50902 ,
        101.79268 ],
       [ 81.84839 ,  83.40687 ,  94.19197 , ...,  81.28216 ,  81.54924 ,
         82.39807 ],
       [ 88.430405,  87.17563 ,  86.038605, ...,  97.72883 ,  94.21663 ,
         90.846855]], dtype=float32)

In [25]:
y_test_true

array([[ 88.52845 ,  86.56636 ,  84.58691 , ...,  87.13936 ,  87.781815,
         88.406906],
       [108.32301 , 107.454834, 106.58665 , ..., 114.9212  , 112.95911 ,
        110.97965 ],
       [ 95.99483 ,  99.64119 , 103.28755 , ...,  87.95545 ,  89.44872 ,
         90.959366],
       ...,
       [113.87938 , 108.49665 , 103.11392 , ...,  91.27191 ,  96.30737 ,
        101.32546 ],
       [ 81.46145 ,  82.972084,  93.56391 , ...,  78.40544 ,  78.457535,
         80.02026 ],
       [ 88.58054 ,  86.913635,  85.229355, ..., 100.11001 ,  96.04691 ,
         92.00118 ]], dtype=float32)

In [79]:
model.save("o3_1dcnn.keras")